In [ ]:
import json

import matplotlib.pyplot as plt

In [ ]:
instruct_eval_loss = []
instruct_train_loss = []
path_to_save_metrics = "../../../results/llama3_results/classification_head/metrics"
steps = []
with open(path_to_save_metrics + "/trainer_state.json") as f:
    instruct_metrics = json.load(f)
    log_history = instruct_metrics["log_history"]

    for log in log_history:
        if "eval_loss" in log:
            instruct_eval_loss.append(log["eval_loss"])
            steps.append(log["step"])
        elif "loss" in log:
            instruct_train_loss.append(log["loss"])

In [ ]:
assert len(instruct_eval_loss) == len(instruct_train_loss), "Length must match"
assert len(instruct_eval_loss) == len(steps), "Length must match"

In [ ]:
# Instruct finetuning metrics
fig, ax = plt.subplots()

ax.plot(steps, instruct_eval_loss, "r-", label="Evaluation loss")
ax.plot(steps, instruct_train_loss, "g-", label="Train loss")
ax.set_xlabel("Steps")
ax.set_ylabel("Loss", color="black")
ax.legend()
plt.title("Llama 3 Head classifier")
txt = "lr=2.0e-06, batch_size=8, epochs=2.5, \n Training duration: 19.84 hours"
plt.figtext(0.5, -0.05, txt, wrap=True, horizontalalignment="center", fontsize=12)

plt.savefig(path_to_save_metrics + "/graph_1_epoch.png")
plt.show()

In [1]:
import torch
from Scripts.llama_model_wrapper import HeadClassifierWrapper, InstructModelWrapper
from Scripts.load_dataset import load_dataset

In [2]:
_, test_ds = load_dataset(
    "../../../../German_newspaper_articles/10kGNAD/train.csv",
    "../../../../German_newspaper_articles/10kGNAD/test.csv",
)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "mps"

# Instruction metrics

In [ ]:
model_kwargs = {
    "path": "../../../results/llama3_results/instruct/model.nosync",
    "tokenizer_path": "meta-llama/Meta-Llama-3-8B-Instruct",
    "torch_dtype": torch.bfloat16,
    "device_map": "auto",
}

In [ ]:
instruct_model_wrapper = InstructModelWrapper(**model_kwargs)

In [ ]:
test_ds = test_ds.map(
    instruct_model_wrapper.create_test_messages, remove_columns=["text"]
)

In [ ]:
test_ds = test_ds.map(instruct_model_wrapper.tokenize_messages)

In [ ]:
test_ds.set_format("torch", device=device)

In [ ]:
correct = 0
correct_dict = {
    "Web": 0,
    "International": 0,
    "Etat": 0,
    "Wirtschaft": 0,
    "Panorama": 0,
    "Sport": 0,
    "Wissenschaft": 0,
    "Kultur": 0,
    "Inland": 0,
}
y_true = []
y_pred = []
wrong = []
instruct_model_wrapper.model.eval()
for i, sample in enumerate(test_ds):
    outputs = instruct_model_wrapper.model.generate(
        sample["input_ids"],
        max_new_tokens=128,
        eos_token_id=instruct_model_wrapper.terminators,
        do_sample=False,
        temperature=0.1,
        top_p=0.9,
    )
    response = outputs[0][sample["input_ids"].shape[-1] :]
    response = instruct_model_wrapper.tokenizer.decode(
        response, skip_special_tokens=True
    )
    y_true.append(sample["label"])
    y_pred.append(response)
    if sample["label"] in response:
        correct += 1
        correct_dict[sample["label"]] += 1
        print(f"At {i}: {response}")

In [ ]:
assert len(set(y_true)) == len(set(y_pred)), "Labels are not the same"

In [ ]:
from collections import Counter

import matplotlib.pyplot as plt

In [ ]:
label_counts = Counter(test_ds["label"])
label_counts

In [ ]:
labels = list(label_counts.keys())
differences = dict()
for label in labels:
    differences[label] = correct_dict[label] / label_counts[label]

In [ ]:
path_to_save_metrics = "../../../results/llama3_results/instruct/metrics"

difference_values = [value * 100 for value in differences.values()]
difference_labels = [value for value in differences.keys()]
fig, ax = plt.subplots()

xs = range(len(difference_labels))
ys = [difference_values[x] for x in xs]

ax.bar(difference_labels, ys, 0.6)
plt.title("correct per category")
plt.xlabel("category")
plt.ylabel("accuracy in %")
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment="right")
plt.savefig(path_to_save_metrics + "/test_evaluation.png")

plt.show()

In [ ]:
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    confusion_matrix,
    f1_score,
)

In [ ]:
cm = confusion_matrix(
    y_true,
    y_pred,
    labels=[
        "Web",
        "International",
        "Etat",
        "Wirtschaft",
        "Panorama",
        "Sport",
        "Wissenschaft",
        "Kultur",
        "Inland",
    ],
)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

In [ ]:
accuracy_score(y_true, y_pred)

In [ ]:
f1_score(y_true, y_pred, average="weighted")

# Classifaction head metrics

In [4]:
model_kwargs = {
    "path": "../../../results/llama3_results/classification_head/model.nosync",
    "num_labels": 9,
    "tokenizer_path": "meta-llama/Meta-Llama-3-8B-Instruct",
    "device_map": "mps",
    "use_cache": False,
    "quantization_config": None,
}

In [5]:
classification_wrapper = HeadClassifierWrapper(**model_kwargs)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/Malte/Documents/Technische Informatik /Forschungsprojekt/TextClassifier/.venv.nosync/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
classification_wrapper.model

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

In [7]:
test_ds = test_ds.map(classification_wrapper.tokenize_messages)

Parameter 'function'=<bound method HeadClassifierWrapper.tokenize_messages of <Scripts.llama_model_wrapper.HeadClassifierWrapper object at 0x15c4ba7d0>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1028 [00:00<?, ? examples/s]

KeyError: 'messages'

In [ ]:
test_ds.set_format("torch", device=device)

In [ ]:
correct = 0
correct_dict = {
    "Web": 0,
    "International": 0,
    "Etat": 0,
    "Wirtschaft": 0,
    "Panorama": 0,
    "Sport": 0,
    "Wissenschaft": 0,
    "Kultur": 0,
    "Inland": 0,
}
y_true = []
y_pred = []
wrong = []
instruct_model_wrapper.model.eval()
for i, sample in enumerate(test_ds):
    output = instruct_model_wrapper.model(sample["input_ids"])

    y_true.append(sample["label"])
    y_pred.append(output)
    if output in sample["label"]:
        correct += 1
        correct_dict[sample["label"]] += 1
        print(f"At {i}: {output}")